In [24]:
! pip install crewai langchain langchain-openai langchain-community markdown


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: C:\Users\Renan\AppData\Local\Programs\Python\Python311\python.exe -m pip install --upgrade pip


In [25]:
import json
import os
import markdown
from crewai import Agent, Task, Crew, Process

from langchain_openai import ChatOpenAI
from langchain_community.tools import DuckDuckGoSearchRun

In [ ]:
os.environ["OPENAI_API_KEY"] = "openeai_key_here"

In [27]:
# Criar ferramenta de pesquisa
from crewai.tools import BaseTool
from pydantic import BaseModel, Field

class SearchInput(BaseModel):
    query: str = Field(description="Termo de pesquisa")

class SearchTool(BaseTool):
    name: str = "Pesquisa Web"
    description: str = "Pesquisa informações na web sobre jogos e tendências"
    args_schema: type[BaseModel] = SearchInput
    
    def _run(self, query: str) -> str:
        from langchain_community.tools import DuckDuckGoSearchRun
        return DuckDuckGoSearchRun().run(query)

search_tool = SearchTool()


In [28]:
# Criar o Agente Roteirista
roteirista = Agent(
    role="Roteirista de Vídeos Gamer",
    goal="""Pesquisar e elaborar um roteiro detalhado para um vídeo completo no YouTube sobre {query}.
    O roteiro deve incluir uma introdução cativante, desenvolvimento do tema com informações relevantes 
    e uma conclusão que incentive o engajamento do público.""",
    backstory="""Você é um roteirista experiente, especializado na criação de roteiros para vídeos de jogos no YouTube.
    Com um profundo conhecimento sobre a indústria de jogos e as tendências atuais, você sabe como capturar a atenção do público desde o início do vídeo.
    
    Você é especialista em criação de conteúdo e storytelling para vídeos de games, sabendo exatamente como estruturar
    narrativas envolventes que prendem a atenção dos espectadores do início ao fim.
    
    Sua habilidade em estruturar o conteúdo de forma lógica e envolvente faz com que os espectadores permaneçam interessados até o final.
    Você também entende a importância de adaptar o tom e o estilo do roteiro ao público-alvo, garantindo que cada vídeo ressoe com os interesses e preferências dos espectadores.""",
    verbose=True,
    llm=llm,
    max_iterations=5,
    allow_delegation=False,
    memory=True
)


In [29]:
# Criar a Tarefa de Elaboração do Roteiro
elaborar_roteiro = Task(
    description="""Pesquise informações atualizadas sobre {query} e elabore um roteiro detalhado e completo para um vídeo no YouTube.
    
    O roteiro deve conter:
    
    1. **INTRODUÇÃO CATIVANTE** (30-45 segundos)
       - Hook inicial que prenda a atenção nos primeiros 5 segundos
       - Apresentação do tema do vídeo
       - Promessa de valor para o espectador
    
    2. **DESENVOLVIMENTO** (corpo principal do vídeo)
       - Informações relevantes e bem pesquisadas sobre o tema
       - Estrutura lógica e progressiva
       - Storytelling envolvente com narrativa clara
       - Exemplos práticos e referências
       - Transições suaves entre os tópicos
    
    3. **CONCLUSÃO** (30-45 segundos)
       - Resumo dos pontos principais
       - Call-to-action para engajamento (like, comentário, inscrição)
       - Gancho para o próximo vídeo
    
    Use a ferramenta de pesquisa para enriquecer o conteúdo com informações atualizadas e relevantes.
    Certifique-se de que o roteiro seja envolvente, informativo e adequado ao público gamer do YouTube.""",
    expected_output="""Um roteiro completo e detalhado formatado em Markdown contendo:
    
    - Título sugerido para o vídeo
    - Introdução com hook inicial
    - Desenvolvimento organizado em seções/tópicos
    - Sugestões de momentos para elementos visuais ou B-roll
    - Conclusão com call-to-action
    - Tempo estimado de duração de cada seção
    - Tom de voz e estilo sugerido
    
    O roteiro deve estar pronto para ser usado pelo criador de conteúdo.""",
    agent=roteirista,
    tools=[search_tool]
)

In [30]:
# Criar o Agente Designer de Thumbnails
designer_thumbnail = Agent(
    role="Designer de Thumbnails para Vídeos Gamer",
    goal="""Utilizar o roteiro gerado para produzir três opções de thumbnails inspiradas no conteúdo do vídeo sobre {query}.
    As thumbnails devem destacar elementos visuais atrativos que maximizem os cliques.""",
    backstory="""Você é um designer gráfico com vasta experiência em criar thumbnails chamativas para YouTube.
    
    Com anos de experiência analisando thumbnails de sucesso no nicho gamer, você domina a arte de combinar:
    - Elementos visuais impactantes que capturam a essência do conteúdo
    - Paletas de cores vibrantes que se destacam no feed do YouTube
    - Tipografia bold e legível que comunica a mensagem em segundos
    - Composição equilibrada que guia o olhar do espectador
    
    Você entende a psicologia por trás do clique e sabe exatamente quais elementos visuais funcionam melhor
    para o público gamer. Suas thumbnails consistentemente alcançam altas taxas de cliques (CTR).""",
    verbose=True,
    llm=llm,
    max_iterations=5,
    allow_delegation=False,
    memory=True
)


In [31]:
# Criar a Tarefa de Criação de Thumbnails
criar_thumbnails = Task(
    description="""Analise cuidadosamente o roteiro do vídeo e crie TRÊS OPÇÕES DISTINTAS de thumbnails.
    
    Cada opção de thumbnail deve ser inspirada no conteúdo do vídeo e incluir:
    
    **Para cada uma das 3 opções:**
    
    1. **Conceito Visual Principal**
       - Elemento central que captura a essência do vídeo
       - Personagens, objetos ou cenas marcantes do tema
    
    2. **Paleta de Cores**
       - Cores vibrantes e contrastantes
       - Esquema de cores que se destaca no feed do YouTube
    
    3. **Texto na Thumbnail**
       - Título curto e impactante (máximo 5-6 palavras)
       - Tamanho de fonte grande e legível
       - Cor que contrasta com o fundo
    
    4. **Elementos de Destaque**
       - Bordas, sombras, ou efeitos de brilho
       - Ícones ou símbolos relevantes ao tema
       - Expressões faciais (se aplicável)
    
    5. **Composição e Layout**
       - Regra dos terços ou composição centralizada
       - Hierarquia visual clara
       - Espaço negativo apropriado
    
    Após criar as 3 opções, analise cada uma e ESCOLHA A MELHOR, justificando sua escolha baseada em:
    - Potencial de CTR (Click-Through Rate)
    - Alinhamento com o conteúdo do vídeo
    - Apelo visual para o público gamer
    - Clareza da mensagem""",
    expected_output="""Um documento formatado em Markdown contendo:
    
    ## OPÇÃO 1
    - **Conceito Visual:** [descrição detalhada]
    - **Paleta de Cores:** [cores específicas]
    - **Texto:** [texto exato a ser usado]
    - **Elementos de Destaque:** [lista de elementos]
    - **Layout:** [descrição da composição]
    
    ## OPÇÃO 2
    - **Conceito Visual:** [descrição detalhada]
    - **Paleta de Cores:** [cores específicas]
    - **Texto:** [texto exato a ser usado]
    - **Elementos de Destaque:** [lista de elementos]
    - **Layout:** [descrição da composição]
    
    ## OPÇÃO 3
    - **Conceito Visual:** [descrição detalhada]
    - **Paleta de Cores:** [cores específicas]
    - **Texto:** [texto exato a ser usado]
    - **Elementos de Destaque:** [lista de elementos]
    - **Layout:** [descrição da composição]
    
    ## THUMBNAIL ESCOLHIDA
    **Opção Selecionada:** [número da opção]
    
    **Justificativa:**
    [explicação detalhada de por que essa opção é a melhor, incluindo análise de CTR potencial e apelo ao público]
    
    **Descrição Final da Thumbnail:**
    [descrição completa e detalhada da thumbnail vencedora, pronta para ser enviada a um designer gráfico]""",
    agent=designer_thumbnail,
    context=[elaborar_roteiro]
)


In [32]:
revisor = Agent(
    role="Revisor de Roteiros e Thumbnails para Vídeos Gamer",
    goal="""Revisar o roteiro e a descrição da thumbnail criada para garantir que ambos estejam alinhados com as melhores práticas de criação de conteúdo no YouTube.
    Fornecer feedback detalhado e sugestões de melhorias.""",
    backstory="""Você é um especialista em criação de conteúdo para YouTube, com vasta experiência em roteiros e design visual.
    
    Com um histórico comprovado de vídeos bem-sucedidos, você entende profundamente o que funciona no nicho gamer.
    Sua expertise inclui:
    - Análise crítica de roteiros para garantir clareza, engajamento e estrutura lógica
    - Avaliação de thumbnails para maximizar CTR através de elementos visuais eficazes
    - Conhecimento das tendências atuais e preferências do público gamer
    
    Você é conhecido por sua atenção aos detalhes e capacidade de fornecer feedback construtivo que eleva a qualidade do conteúdo.""",
    verbose=True,
    llm=llm,
    max_iterations=5,
    allow_delegation=False,
    memory=True
)

In [33]:
# Criar a Tarefa de Revisão Final
revisar_e_consolidar = Task(
    description="""Revise cuidadosamente o roteiro e a thumbnail criada, e produza uma versão final consolidada.
    
    **Sua revisão deve incluir:**
    
    1. **REVISÃO DO ROTEIRO**
       - Correção de erros gramaticais e ortográficos
       - Verificação da fluidez e clareza do texto
       - Análise do timing e duração de cada seção
       - Sugestões de melhorias na narrativa ou estrutura (se necessário)
       - Verificação de que o roteiro está completo e pronto para gravação
    
    2. **REVISÃO DA THUMBNAIL**
       - Análise da thumbnail escolhida
       - Verificação do alinhamento com o conteúdo do roteiro
       - Avaliação do potencial de CTR
       - Sugestões de ajustes finais (se necessário)
    
    3. **CONSOLIDAÇÃO FINAL**
       - Criar um documento único e profissional
       - Combinar roteiro revisado + descrição da thumbnail
       - Incluir todas as informações necessárias para produção
       - Apresentar de forma clara e organizada
    
    O documento final deve estar 100% pronto para que o criador de conteúdo possa:
    - Gravar o vídeo seguindo o roteiro
    - Encomendar a thumbnail com o designer gráfico""",
    expected_output="""Um documento final completo formatado em Markdown contendo:
    
    # PACOTE COMPLETO DE PRODUÇÃO - {query}
    
    ---
    
    ## 📋 INFORMAÇÕES DO PROJETO
    - **Tema:** [tema do vídeo]
    - **Duração Estimada:** [tempo total]
    - **Público-Alvo:** [descrição]
    - **Data de Revisão:** [data]
    
    ---
    
    ## 🎬 ROTEIRO DO VÍDEO
    
    ### Título do Vídeo
    [título final revisado]
    
    ### Introdução 
    [roteiro revisado e polido]
    
    ### Desenvolvimento
    [roteiro completo revisado com todas as seções]
    
    ### Conclusão
    [roteiro da conclusão revisado]
    
    ### ⏱️ Timing Total
    [breakdown de tempo de cada seção]
    
    ---
    
    ## 🎨 THUMBNAIL DO VÍDEO
    
    ### Thumbnail Selecionada
    **Opção:** [número da opção escolhida]
    
    ### Descrição Detalhada para Designer
    [descrição completa da thumbnail com todos os elementos visuais]
    
    ### Elementos Visuais
    - **Conceito:** [descrição]
    - **Cores:** [paleta de cores]
    - **Texto:** [texto exato]
    - **Layout:** [composição]
    - **Efeitos:** [detalhes de efeitos visuais]
    
    ---
    
    ## ✅ CHECKLIST DE PRODUÇÃO
    - [ ] Roteiro revisado e aprovado
    - [ ] Thumbnail especificada
    - [ ] Material de pesquisa incluído
    - [ ] Timing verificado
    - [ ] Pronto para gravação
    
    ---
    
    ## 📝 NOTAS DA REVISÃO
    [comentários do revisor sobre pontos fortes do conteúdo e quaisquer observações importantes]""",
    agent=revisor,
    context=[elaborar_roteiro, criar_thumbnails],
)


In [34]:
crew = Crew(
        agents=[roteirista, designer_thumbnail, revisor],
        tasks=[elaborar_roteiro, criar_thumbnails, revisar_e_consolidar],
        verbose=True,
        process=Process.sequential,
        full_output= True,
        share_crew = False,
        manager_llm= llm,
        max_iter= 15
)

In [35]:
result = crew.kickoff(inputs={'query': 'Melhores jogos de 2020'})

In [36]:
result.raw

'```markdown\n# PACOTE COMPLETO DE PRODUÇÃO - Melhores Jogos de 2020\n\n---\n\n## 📋 INFORMAÇÕES DO PROJETO\n- **Tema:** Os Melhores Jogos de 2020 que Você Não Pode Perder!\n- **Duração Estimada:** 8 minutos e 30 segundos\n- **Público-Alvo:** Gamers interessados em jogos populares e premiados de 2020\n- **Data de Revisão:** Outubro de 2023\n\n---\n\n## 🎬 ROTEIRO DO VÍDEO\n\n### Título do Vídeo\nOs Melhores Jogos de 2020 que Você Não Pode Perder!\n\n### Introdução \n"Você está pronto para reviver os momentos mais épicos de 2020? Prepare-se para uma viagem pelos jogos que marcaram o ano e redefiniram o conceito de diversão! Hoje, vamos explorar os títulos que dominaram consoles e PCs em 2020, trazendo inovações, histórias emocionantes e gráficos de cair o queixo. Neste vídeo, você vai descobrir os detalhes por trás dos jogos mais aclamados e entender por que eles são amados por milhões de jogadores ao redor do mundo."\n\n### Desenvolvimento\n\n1. **The Last of Us Part II** \n   - Descriçã

In [37]:
from IPython.display import Markdown
Markdown(result.raw)

```markdown
# PACOTE COMPLETO DE PRODUÇÃO - Melhores Jogos de 2020

---

## 📋 INFORMAÇÕES DO PROJETO
- **Tema:** Os Melhores Jogos de 2020 que Você Não Pode Perder!
- **Duração Estimada:** 8 minutos e 30 segundos
- **Público-Alvo:** Gamers interessados em jogos populares e premiados de 2020
- **Data de Revisão:** Outubro de 2023

---

## 🎬 ROTEIRO DO VÍDEO

### Título do Vídeo
Os Melhores Jogos de 2020 que Você Não Pode Perder!

### Introdução 
"Você está pronto para reviver os momentos mais épicos de 2020? Prepare-se para uma viagem pelos jogos que marcaram o ano e redefiniram o conceito de diversão! Hoje, vamos explorar os títulos que dominaram consoles e PCs em 2020, trazendo inovações, histórias emocionantes e gráficos de cair o queixo. Neste vídeo, você vai descobrir os detalhes por trás dos jogos mais aclamados e entender por que eles são amados por milhões de jogadores ao redor do mundo."

### Desenvolvimento

1. **The Last of Us Part II** 
   - Descrição sobre a sua narrativa envolvente e prêmios recebidos.
   - Mostrar cenas icônicas e emocionantes do jogo.
   
2. **Hades**
   - Destaque para sua narrativa rica e jogabilidade viciante.
   - Mostrar cenas de gameplay frenético e batalhas contra chefes.
   
3. **Ghost of Tsushima**
   - Comentar sobre sua estética bela e narrativa cativante.
   - Apresentar mundo aberto e cenários do Japão feudal.
   
4. **Animal Crossing: New Horizons**
   - Apreciação por se tornar um fenômeno cultural.
   - Mostrar personalizações de ilha e interações de personagens.
   
5. **Final Fantasy VII Remake**
   - Celebrar o remake aguardado que trouxe nova vida à uma clássica história.
   - Exibir combates e cenas cinematográficas do jogo.

### Conclusão
"Neste vídeo, revisitamos os jogos que mais se destacaram em 2020, cada um oferecendo experiências únicas que nos ligaram a mundos e histórias inesquecíveis. Qual foi o seu jogo favorito de 2020? Deixe seu comentário abaixo, não esqueça de dar um like e se inscrever para mais conteúdo gamer! Quer saber o que 2021 nos reserva no mundo dos games? Fique ligado no nosso próximo vídeo onde exploraremos lançamentos imperdíveis!"

### ⏱️ Timing Total
- Introdução: 45 segundos
- Desenvolvimento: 7 minutos
- Conclusão: 45 segundos

---

## 🎨 THUMBNAIL DO VÍDEO

### Thumbnail Selecionada
**Opção:** 1

### Descrição Detalhada para Designer
A thumbnail apresenta uma montagem circular dos cinco jogos destacados, com Ellie de "The Last of Us Part II" no centro, cercada por personagens e cenas dos outros jogos no arco externo. As cores vibrantes e contrastantes de laranja, roxo e verde criam um visual dinâmico, enquanto um fundo azul escuro acentua o foco na montagem. O texto "TOP JOGOS 2020!" se destaca em fonte bold, garantindo clareza em uma rápida visualização. Efeitos sutis de sombras e bordas iluminadas em cada personagem acrescentam à impressão de um pôster elaborado.

### Elementos Visuais
- **Conceito:** Montagem dos cinco jogos com "The Last of Us Part II" no centro.
- **Cores:** Laranja, roxo e verde sobre fundo azul escuro.
- **Texto:** "TOP JOGOS 2020!"
- **Layout:** Circular, com uso da regra dos terços.
- **Efeitos:** Sombras sutis e bordas iluminadas nos personagens.

---

## ✅ CHECKLIST DE PRODUÇÃO
- [x] Roteiro revisado e aprovado
- [x] Thumbnail especificada
- [ ] Material de pesquisa incluído
- [x] Timing verificado
- [x] Pronto para gravação

---

## 📝 NOTAS DA REVISÃO
A estrutura narrativa é clara, cobrindo de maneira eficaz o valor e o impacto dos jogos discutidos em 2020. A introdução e conclusão estão bem amarradas com elementos de engajamento ao público. Além disso, a opção de thumbnail selecionada complementa perfeitamente os jogos discutidos no vídeo, promovendo um bom potencial de clique graças ao seu design vibrante e atrativo. Recomendo a inclusão de música de fundo envolvente durante as seções de B-roll para manter o público engajado.
```